In [9]:
import cv2
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern, hog
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


# **Build Training Data**

In [ ]:
df_doulbe = pd.read_csv("./Result/Metadata_Optics_doulbe.csv")
df_doulbe.head()

In [ ]:
df_ppm = pd.read_csv("./Result/Metadata_Optics_ppm.csv")
df_ppm.head()

In [12]:
# Set the image_id as the index in data_pd
df_doulbe.set_index('ID_img', inplace=True)
df_ppm.set_index('ID_img', inplace=True)

In [ ]:
# Path to the directory containing data
data_dir = './Result/middles/'
image_size = (224, 224)  # Image size (optional)

# Read all images from the directory
def load_images(data_dir, image_size):
    images = []
    labels = []
    for category in os.listdir(data_dir):
        path = os.path.join(data_dir, category)
        
        for img_name in os.listdir(path):
            try:
                img_path = os.path.join(path, img_name)
                image = cv2.imread(img_path)
                image = cv2.resize(image, image_size)
                if category.__contains__("double"):
                    label = df_doulbe.loc[img_name, 'ppm']  
                else:
                    label = df_ppm.loc[img_name, 'ppm']
                images.append(image)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")

    return np.array(images), np.array(labels)

# Load dataset
images, labels = load_images(data_dir, image_size)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.4, random_state=42)

# **Data Preprocessing**

***Extract Color Features***

In [ ]:
# Convert color space to RGB, HSV, L*a*b*
def extract_color_features(image):
    rgb_mean = np.mean(image, axis=(0,1))  # Compute mean per channel in RGB space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_mean = np.mean(hsv_image, axis=(0,1))  # Compute mean in HSV space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    lab_mean = np.mean(lab_image, axis=(0,1))  # Compute mean in L*a*b* space
    
    return np.concatenate([rgb_mean, hsv_mean, lab_mean])

# Apply color feature extraction to all images
X_train_features = np.array([extract_color_features(img) for img in X_train])
X_test_features = np.array([extract_color_features(img) for img in X_test])

***Extract Texture Features***

In [ ]:
# Extract LBP and HOG
def extract_texture_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # LBP (Local Binary Pattern)
    lbp = local_binary_pattern(gray_image, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp, bins=np.arange(0, 11), range=(0, 10), density=True)
    
    # HOG (Histogram of Oriented Gradients)
    hog_features, _ = hog(gray_image, orientations=9, pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2), visualize=True)
    
    return np.concatenate([lbp_hist, hog_features])

# Apply texture feature extraction
X_train_texture = np.array([extract_texture_features(img) for img in X_train])
X_test_texture = np.array([extract_texture_features(img) for img in X_test])

In [ ]:
# Combine color and texture features
X_train_combined = np.concatenate([X_train_features, X_train_texture], axis=1)
X_test_combined = np.concatenate([X_test_features, X_test_texture], axis=1)

In [ ]:
print(X_train_combined.shape)
print(X_test_combined.shape)

# **Train Prediction Model**

In [ ]:
# Linear Regression (for regression tasks)
pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),  # Standardize the data
    ('lr', LinearRegression())
])

param_grid_lr = {
    'lr__fit_intercept': [True, False],
}

In [ ]:
# SVM (Support Vector Regressor) for regression tasks
pipeline_svr = Pipeline([
    ('scaler', StandardScaler()),  # Standardize the data
    ('svr', SVR())
])

param_grid_svr = {
    'svr__kernel': ['linear', 'rbf'],
    'svr__C': [0.1, 1, 10],
    'svr__gamma': ['scale', 'auto']
}

In [ ]:
# Random Forest Regressor (for regression tasks)
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),  # Standardize the data
    ('rf', RandomForestRegressor())
])

param_grid_rf = {
    'rf__n_estimators': [50, 100, 200],  # Number of trees
    'rf__max_depth': [None, 10, 20],  # Maximum depth of the trees
    'rf__min_samples_split': [2, 5, 10]  # Minimum samples required to split a node
}

In [ ]:
# GridSearchCV for each model
grid_search_lr = GridSearchCV(pipeline_lr, param_grid_lr, cv=5, scoring='neg_mean_squared_error')
grid_search_svr = GridSearchCV(pipeline_svr, param_grid_svr, cv=5, scoring='neg_mean_squared_error')
grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=5, scoring='neg_mean_squared_error')

# Train each model
print("Training Linear Regression...")
grid_search_lr.fit(X_train_combined, y_train)

print("Training SVR...")
grid_search_svr.fit(X_train_combined, y_train)

print("Training Random Forest Regressor...")
grid_search_rf.fit(X_train_combined, y_train)

# Print the best parameters for each model
print("Best parameters for Linear Regression:", grid_search_lr.best_params_)
print("Best parameters for SVR:", grid_search_svr.best_params_)
print("Best parameters for Random Forest Regressor:", grid_search_rf.best_params_)

In [ ]:
# Save the best model and parameters of Linear Regression
joblib.dump(grid_search_lr.best_estimator_, 'best_linear_regression_model.pkl')
joblib.dump(grid_search_lr.best_params_, 'best_linear_regression_params.pkl')

# Save the best model and parameters of SVR
joblib.dump(grid_search_svr.best_estimator_, 'best_svr_model.pkl')
joblib.dump(grid_search_svr.best_params_, 'best_svr_params.pkl')

# Save the best model and parameters of Random Forest Regressor
joblib.dump(grid_search_rf.best_estimator_, 'best_random_forest_model.pkl')
joblib.dump(grid_search_rf.best_params_, 'best_random_forest_params.pkl')

In [ ]:
# # Load the saved model and parameters of Linear Regression
# best_lr_model = joblib.load('best_linear_regression_model.pkl')
# best_lr_params = joblib.load('best_linear_regression_params.pkl')

# # Load the saved model and parameters of SVR
# best_svr_model = joblib.load('best_svr_model.pkl')
# best_svr_params = joblib.load('best_svr_params.pkl')

# # Load the saved model and parameters of Random Forest
# best_rf_model = joblib.load('best_random_forest_model.pkl')
# best_rf_params = joblib.load('best_random_forest_params.pkl')

# # Print the best parameters
# print("Best parameters for Linear Regression:", best_lr_params)
# print("Best parameters for SVR:", best_svr_params)
# print("Best parameters for Random Forest Regressor:", best_rf_params)

In [ ]:
# Evaluate the model on the test set
y_pred_lr = grid_search_lr.predict(X_test_combined)
y_pred_svr = grid_search_svr.predict(X_test_combined)
y_pred_rf = grid_search_rf.predict(X_test_combined)

# Calculate Mean Squared Error (MSE) for Linear Regression, SVR, and Random Forest Regressor
mse_lr = mean_squared_error(y_test, y_pred_lr)
mse_svr = mean_squared_error(y_test, y_pred_svr)
mse_rf = mean_squared_error(y_test, y_pred_rf)

# Calculate R-squared score (R²) for each model
r2_lr = r2_score(y_test, y_pred_lr)
r2_svr = r2_score(y_test, y_pred_svr)
r2_rf = r2_score(y_test, y_pred_rf)

# Print evaluation results
print(f"Linear Regression MSE: {mse_lr}, LR R²: {r2_lr}\n")
print(f"SVR MSE: {mse_svr}, SVR R²: {r2_svr}\n")
print(f"Random Forest Regressor MSE: {mse_rf}, RF R²: {r2_rf}\n")

In [ ]:

# Data for MSE and R^2
models = ['Linear Regression','SVR', 'Random Forest Regressor']
mse_values = [mse_lr, mse_svr, mse_rf]
r2_values = [r2_lr, r2_svr, r2_rf]

# Create MSE bar chart
plt.figure(figsize=(14, 6))

# MSE
plt.subplot(1, 2, 1)
plt.bar(models, mse_values, color=['blue', 'green', 'red'])
plt.title('Mean Squared Error (MSE) of Models')
plt.ylabel('MSE')
plt.xticks(rotation=15)
for i, v in enumerate(mse_values):
    plt.text(i, v + 0.01, f"{v:.4f}", ha='center', fontsize=12)

# R^2
plt.subplot(1, 2, 2)
plt.bar(models, r2_values, color=['blue', 'green', 'red'])
plt.title('R^2 of Models')
plt.ylabel('R^2')
plt.xticks(rotation=15)
for i, v in enumerate(r2_values):
    plt.text(i, v + 0.01, f"{v:.4f}", ha='center', fontsize=12)

# Display the plot
plt.tight_layout()
plt.show()